Using the Churn Modelling csv and transforming it then saving it into the bucket

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Read_CSV_from_GCS") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 16:36:31 INFO SparkEnv: Registering MapOutputTracker
24/03/29 16:36:31 INFO SparkEnv: Registering BlockManagerMaster
24/03/29 16:36:31 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/03/29 16:36:31 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
gcs_uri = "gs://pysparkbucket-jm/Churn_Modelling.csv"

In [4]:
df = spark.read.csv(gcs_uri, header=True, inferSchema=True)

In [5]:
df.show(15)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|

In [6]:
df.select("Geography").distinct().show()

+---------+
|Geography|
+---------+
|  Germany|
|    Spain|
|   France|
+---------+



In [7]:
df.select("Age").distinct().show()

+---+
|Age|
+---+
| 65|
| 81|
| 76|
| 92|
| 64|
| 37|
| 61|
| 35|
| 60|
| 33|
| 83|
| 66|
| 74|
| 36|
| 31|
| 85|
| 53|
| 34|
| 44|
| 57|
+---+
only showing top 20 rows



Perfroming an operation of people in france between the age of 30 to 65

In [8]:
# Filter the data based on the specified conditions
filtered_data = df.filter((df["Geography"] == "France") & (df["Age"].between(30, 65)))


In [9]:
selected_data = filtered_data.select("CreditScore", "Geography", "Gender", "Age", "Balance", "EstimatedSalary")


In [10]:
selected_data.show()

+-----------+---------+------+---+---------+---------------+
|CreditScore|Geography|Gender|Age|  Balance|EstimatedSalary|
+-----------+---------+------+---+---------+---------------+
|        619|   France|Female| 42|      0.0|      101348.88|
|        502|   France|Female| 42| 159660.8|      113931.57|
|        699|   France|Female| 39|      0.0|       93826.63|
|        822|   France|  Male| 50|      0.0|        10062.8|
|        501|   France|  Male| 44|142051.07|        74940.5|
|        528|   France|  Male| 31|102016.72|       80181.12|
|        476|   France|Female| 34|      0.0|       26260.98|
|        732|   France|  Male| 41|      0.0|      170886.17|
|        669|   France|  Male| 46|      0.0|        8487.75|
|        846|   France|Female| 38|      0.0|      187616.16|
|        571|   France|  Male| 44|      0.0|       38433.35|
|        533|   France|  Male| 36|  85311.7|      156731.91|
|        475|   France|Female| 45|134264.04|       27822.99|
|        850|   France| 

Creating the table and saving the table in spark.sql

In [11]:
table_path = "gs://pysparkbucket-jm/France30to65"
selected_data.createOrReplaceTempView("France30to65")

In [13]:
schema = """
    CreditScore INT,
    Geography STRING,
    Gender STRING,
    Age INT,
    Balance DOUBLE,
    EstimatedSalary DOUBLE
"""


In [14]:
# Save the DataFrame as a table using Spark SQL
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS France30to65
    USING parquet
    OPTIONS (
        path '{table_path}'
    )
    AS SELECT * FROM France30to65
""")

24/03/29 16:44:49 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [15]:
spark.sql("SELECT * FROM France30to65").show()

+-----------+---------+------+---+---------+---------------+
|CreditScore|Geography|Gender|Age|  Balance|EstimatedSalary|
+-----------+---------+------+---+---------+---------------+
|        619|   France|Female| 42|      0.0|      101348.88|
|        502|   France|Female| 42| 159660.8|      113931.57|
|        699|   France|Female| 39|      0.0|       93826.63|
|        822|   France|  Male| 50|      0.0|        10062.8|
|        501|   France|  Male| 44|142051.07|        74940.5|
|        528|   France|  Male| 31|102016.72|       80181.12|
|        476|   France|Female| 34|      0.0|       26260.98|
|        732|   France|  Male| 41|      0.0|      170886.17|
|        669|   France|  Male| 46|      0.0|        8487.75|
|        846|   France|Female| 38|      0.0|      187616.16|
|        571|   France|  Male| 44|      0.0|       38433.35|
|        533|   France|  Male| 36|  85311.7|      156731.91|
|        475|   France|Female| 45|134264.04|       27822.99|
|        850|   France| 